Exploiting the `datetime` index, we can extract some new columns containing the information about the day and the month.

In [ ]:
def create_features(
    dataframe,
    features = ["day", "month"],
    weekends = False,
):
    """Generates time-series features out of a dataframe. The index MUST be a DateTimeIndex.

    Args:
        dataframe ([type]):
            DateTimeIndex-ed Pandas DataFrame.
        features (list, optional):
            The number of features to obtain. Defaults to ["day", "month", "year"].
            Possible values: ["hour", "day", "month", "week", "year", "day_name", "month_name"]

        weekends (bool, optional): creates another column with True if the weekday is a . Defaults to False.
    
    Returns the same dataframe, but with new columns
    """

    if "hour" in features:
        dataframe["hour"] = dataframe.index.hour

    if weekends and "day" not in features or "day_names" not in features:
        raise Exception("`weekends` must be chosen with either `day` or `day_names` in `features`, or both")

    if "day" in features:
        dataframe["day"] = dataframe.index.day
        
        if weekends:
            dataframe["is_weekend"] = dataframe["day"] \
                .apply(lambda x: "True" if x > 5 else "False") \
                .astype("bool")

    if "day_names" in features:
        dataframe["day_name"] = dataframe.index.day_name()

        if weekends:
            weekend_days = ["Saturday", "Sunday"]
            dataframe["is_weekend"] = dataframe["day"] \
                .apply(lambda x: "True" if x in weekend_days else "False") \
                .astype("bool")

    if "week" in features:
        dataframe["week"] = dataframe.index.iso

    if "month" in features:
        dataframe["month"] = dataframe.index.month

    if "month_name" in features:
        dataframe["month_name"] = dataframe.index.month_name()
    
    if "year" in features:
        dataframe["year"] = dataframe.index.year

    return dataframe


In [56]:

# def daily_outflow_barchart(dataframe):
    
#     df = dataframe.copy()

#     df.day = dataframe.index.isocalendar().day



daily_outflow["day"] = daily_outflow.index.isocalendar().day
daily_outflow.loc[ daily_outflow.day <= 5, "is_weekend"] = "no"
daily_outflow.loc[ daily_outflow.day > 5, "is_weekend"] = "yes"


fig = daily_outflow.plot.bar(
    x = daily_outflow.index,
    y = "count",
    title="Bikes Daily Outflow (2019)",
    color = "is_weekend",
    template = "none", # for plotly themes: https://plotly.com/python/templates/
    labels = {
        "value": "Number of trips",
        "giorno_partenza" : "date",
        "is_weekend": "weekend"
    }
)

# see here: https://plotly.com/python/reference/layout/
fig.update_layout(
    title = {
        "x" : 0.05,
        # basically, what part of the title to put at position "x"
        "xanchor": "left"
    }
)

fig.show()